In [ ]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
import json

from prompt import COORDINATOR_PROMPT, DEVELOPER_PROMPT, EXECUTE_DEVELOPER_PROMPT
load_dotenv()

client = OpenAI()

class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})

    def get_schema(self):
        return self.function
        
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        name_agent: Optional[str] = None,
        description: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)

        self.name_agent = name_agent
        self.description = description

        self.function={
            "name": self.name_agent,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "user_prompt": {
                        "type": "string",
                        "description": "User prompt to the agent"
                    }
                },
                "required": ["user_query"]
            }
        }
    
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        
        chat_completion = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
        )
        response=chat_completion.choices[0].message.content
        
        self.add_to_memory("assistant", response)
        
        return response

class ManagerAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        list_agents: Optional[list] = None
    ) -> None:
        super().__init__(model,system_prompt)
        
        self.list_agents=list_agents
    
    
    def get_agent_list_schemas(self):
        return [agent.get_schema() for agent in self.list_agents]   
    
    def run(self,prompt):
        
        self.add_to_memory("user", f"[PROMPT]\n{prompt}\n\n[AGENTS]\n{self.get_agent_list_schemas()}")
        
        #print(self.memory)
        
        #Create a plan to resolve the problem with the agents
        chat_completion = client.chat.completions.create(
            messages=self.memory,
            model=self.model
        )
        plan_response=chat_completion.choices[0].message.content
        print(plan_response)
        
        # self.add_to_memory("user", response)
        
        # Get plan in json
        json_plan_response=json.loads(plan_response)
        
        # Execute plan
        for task in json_plan_response:
            print(task)
            agent_to_execute = [agent for agent in self.list_agents if agent.name_agent == task['assigned_to']]
            print(agent_to_execute.name_agent)
            response_agent=agent_to_execute[0].run(task['description'])
            print(response_agent)

        #self.add_to_memory("assistant", response_agent)

        #Check what agent i have to execute to resolve the problem
        chat_completion = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
            functions=self.get_agent_list_schemas()
        )
        print(chat_completion.dict())
        response=chat_completion.choices[0].message.function_call
        
        # #get functional call response
        # agent_name=chat_completion.choices[0]['function']['name']
        # agent_prompt=chat_completion.choices[0]['function']['user_prompt']
        
        # print(agent_name)
        # print(agent_prompt)
        
        # #get the agent to execute
        # agent_to_execute = [agent for agent in self.list_agents if agent.name_agent == agent_name]
        # print(agent_to_execute)
        
        # #Execute the agent
        # response_agent=agent_to_execute.run(agent_prompt)
        
        # print(response_agent)
        
        # self.add_to_memory("assistant", response_agent)
        
        return response
    
agent_a=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="CodeDeveloper",
    description="Agent to resolve developer problems",
    system_prompt=DEVELOPER_PROMPT
)
agent_b=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="UIDesignDeveloper",
    description="Agent to resolve UI design problems",
    system_prompt=EXECUTE_DEVELOPER_PROMPT
)

# Manager Agents
manager=ManagerAgent(
    model="gpt-4-1106-preview",
    list_agents=[agent_a,agent_b],
    system_prompt=COORDINATOR_PROMPT
)

print(manager.run("Create a code for tetris game"))